# Task 1: Introduction

---

For this project, we are going to work on evaluating price of houses given the following features:

1. Year of sale of the house
2. The age of the house at the time of sale
3. Distance from city center
4. Number of stores in the locality
5. The latitude
6. The longitude

![Regression](gambar/regression.png)

Note: This notebook uses `python 3` and these packages: `tensorflow`, `pandas`, `matplotlib`, `scikit-learn`.

## 1.1: Importing Libraries & Helper Functions

First of all, we will need to import some libraries and helper functions. This includes TensorFlow and some utility functions that I've written to save time.

Kode di bawah ini digunakan karena ketika langsung import modul, muncul <a href="https://stackoverflow.com/questions/54843067/no-module-named-torch">'no module named torch'</a>

In [1]:
conda install pytorch-cpu torchvision-cpu -c pytorch

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::pandas==1.0.1=py36h0573a6f_0
  - defaults/noarch::jupyterlab==1.2.6=pyhf63ae98_0
  - defaults/linux-64::scikit-learn==0.22.1=py36hd81dba3_0
  - defaults/linux-64::python-language-server==0.31.7=py36_0
  - defaults/linux-64::bkcharts==0.2=py36_0
  - defaults/linux-64::nb_conda==2.2.1=py36_0
  - defaults/noarch::numpydoc==0.9.2=py_0
  - defaults/linux-64::pytest-arraydiff==0.3=py36h39e3cac_0
  - defaults/linux-64::bottleneck==1.3.2=py36heb32a55_0
  - defaults/linux-64::pywavelets==1.1.1=py36h7b6447c_0
  - defaults/noarch::pytest-astropy==0.8.0=py_0
  - defaults/linux-64::numexpr==2.7.1=py36h423224d_0
  - defaults/noarch::anaconda-project==0.8.4=py_0
  - defaults/linux-64::nbconvert==5.6.1=py36_0
  - defaults/linux-64::h5py==2.10.0=py36h7918eee_0
  - defaults/linux-64::bokeh==1.4.0=py36_0
  - defaults/noarch::jupyter

Kode di bawah ini digunakan karena ketika langsung import modul, muncul <a href="https://stackoverflow.com/questions/41415629/importerror-no-module-named-tensorflow-python">'no module named tensorflow'</a>

In [3]:
pip uninstall tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install tensorflow

     |████████████████████████████████| 394.7 MB 3.0 kB/s  eta 0:00:01    |█▏                              | 14.6 MB 11.0 MB/s eta 0:00:35     |█████▊                          | 70.2 MB 59.2 MB/s eta 0:00:06
     |████████████████████████████████| 42 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 127 kB 70.9 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 38.9 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 51.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 57.3 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 44.1 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 459 kB/s  eta 0:00:01
     |████████████████████████████████| 779 kB 48.6 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 114 kB 59.6 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 64.6 MB/s eta 0:00:01
     |████████████████████████████

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from utils import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

%matplotlib inline
# tf.logging.set_verbosity(tf.logging.ERROR) # can not running

print('Libraries imported.')

Libraries imported.


# Task 2: Importing the Data

## 2.1: Importing the Data

The dataset is saved in a `data.csv` file. We will use `pandas` to take a look at some of the rows.

Data tidak memiliki nama jadi perlu diberi nama <a href="https://www.kite.com/python/answers/how-to-set-column-names-when-importing-a-csv-into-a-pandas-dataframe-in-python#:~:text=Call%20pandas.,order%20they%20appear%20in%20names%20.">'source'</a>

In [10]:
# create column's name
column_names = ["serial", "date", "age", "distance", "stores", "latitude", "longitude", "price"]

In [11]:
df = pd.read_csv('data.csv', names = column_names) # can not running
df.head()

,serial,date,age,distance,stores,latitude,longitude,price
0,0,2009,21,9,6,84,121,14264
1,1,2007,4,2,3,86,121,12032
2,2,2016,18,3,7,90,120,13560
3,3,2002,13,2,2,80,128,12029
4,4,2014,25,5,8,81,122,14157


## 2.2: Check Missing Data

It's a good practice to check if the data has any missing values. In real world data, this is quite common and must be taken care of before any data pre-processing or model training.

In [12]:
df.isna().sum()

serial       0
date         0
age          0
distance     0
stores       0
latitude     0
longitude    0
price        0
dtype: int64

# Task 3: Data Normalization

## 3.1: Data Normalization

We can make it easier for optimization algorithms to find minimas by normalizing the data before training a model.

In [13]:
df = df.iloc[:, 1:]
df_norm = (df - df.mean())/df.std()
df_norm.head()

,date,age,distance,stores,latitude,longitude,price
0,0.015978,0.181384,1.257002,0.345224,-0.307212,-1.260799,0.350088
1,-0.350485,-1.319118,-0.930610,-0.609312,0.325301,-1.260799,-1.836486
2,1.298598,-0.083410,-0.618094,0.663402,1.590328,-1.576456,-0.339584
3,-1.266643,-0.524735,-0.930610,-0.927491,-1.572238,0.948803,-1.839425
4,0.932135,0.534444,0.006938,0.981581,-1.255981,-0.945141,0.245266


## 3.2: Convert Label Value

Because we are using normalized values for the labels, we will get the predictions back from a trained model in the same distribution. So, we need to convert the predicted values back to the original distribution if we want predicted prices.

In [14]:
y_mean = df['price'].mean()
y_std = df['price'].std()

def convert_label_value(pred):
    return int(pred * y_std + y_mean)

print(convert_label_value(0.350088))

14263


# Task 4: Create Training and Test Sets

## 4.1: Select Features

Make sure to remove the column __price__ from the list of features as it is the label and should not be used as a feature.

In [15]:
x = df_norm.iloc[:, :6]

In [16]:
x.head()

,date,age,distance,stores,latitude,longitude
0,0.015978,0.181384,1.257002,0.345224,-0.307212,-1.260799
1,-0.350485,-1.319118,-0.930610,-0.609312,0.325301,-1.260799
2,1.298598,-0.083410,-0.618094,0.663402,1.590328,-1.576456
3,-1.266643,-0.524735,-0.930610,-0.927491,-1.572238,0.948803
4,0.932135,0.534444,0.006938,0.981581,-1.255981,-0.945141


## 4.2: Select Labels

In [17]:
y = df_norm.iloc[:, -1]

In [18]:
y.head()

0    0.350088
1   -1.836486
2   -0.339584
3   -1.839425
4    0.245266
Name: price, dtype: float64

## 4.3: Feature and Label Values

We will need to extract just the numeric values for the features and labels as the TensorFlow model will expect just numeric values as input.

In [19]:
x_arr = x.values
y_arr = y.values

print('features array shape:', x_arr.shape)
print('labels array shape:', y_arr.shape)

features array shape: (5000, 6)
labels array shape: (5000,)


## 4.4: Train and Test Split

We will keep some part of the data aside as a __test__ set. The model will not use this set during training and it will be used only for checking the performance of the model in trained and un-trained states. This way, we can make sure that we are going in the right direction with our model training.

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x_arr, y_arr, test_size = 0.05, 
                                                   random_state=0)

print('Training set:', x_train.shape, y_train.shape)
print('Test set:', x_test.shape, y_test.shape)

Training set: (4750, 6) (4750,)
Test set: (250, 6) (250,)


# Task 5: Create the Model

## 5.1: Create the Model

Let's write a function that returns an untrained model of a certain architecture.

In [21]:
def get_model():
    model = Sequential([
        Dense(10, input_shape = (6,), activation='relu'),
        Dense(20, activation='relu'),
        Dense(5, activation='relu'),
        Dense(1)
    ])
    model.compile(
        loss='mse',
        optimizer='adam'
    )
    return model

get_model().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                70        
_________________________________________________________________
dense_1 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 105       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


# Task 6: Model Training

## 6.1: Model Training

We can use an `EarlyStopping` callback from Keras to stop the model training if the validation loss stops decreasing for a few epochs.

In [22]:
es_cb = EarlyStopping(monitor='val_loss', patience=5)

model = get_model()
preds_on_untrained = model.predict(x_test)

history = model.fit(x_train, y_train,
                   validation_data = (x_test, y_test),
                   epochs = 100,
                   callbacks = [es_cb]
                   )

Epoch 1/100
149/149 [==============================] - 1s 3ms/step - loss: 1.2363 - val_loss: 0.4727
Epoch 2/100
149/149 [==============================] - 0s 1ms/step - loss: 0.4393 - val_loss: 0.2381
Epoch 3/100
149/149 [==============================] - 0s 1ms/step - loss: 0.2480 - val_loss: 0.1863
Epoch 4/100
149/149 [==============================] - 0s 1ms/step - loss: 0.1985 - val_loss: 0.1651
Epoch 5/100
149/149 [==============================] - 0s 1ms/step - loss: 0.1749 - val_loss: 0.1529
Epoch 6/100
149/149 [==============================] - 0s 1ms/step - loss: 0.1639 - val_loss: 0.1509
Epoch 7/100
149/149 [==============================] - 0s 1ms/step - loss: 0.1609 - val_loss: 0.1459
Epoch 8/100
149/149 [==============================] - 0s 1ms/step - loss: 0.1545 - val_loss: 0.1468
Epoch 9/100
149/149 [==============================] - 0s 1ms/step - loss: 0.1551 - val_loss: 0.1448
Epoch 10/100
149/149 [==============================] - 0s 1ms/step - loss: 0.1589 - val_lo

## 6.2: Plot Training and Validation Loss

Let's use the `plot_loss` helper function to take a look training and validation loss.

In [23]:
plot_loss(history) # can not running

NameError: name 'plot_loss' is not defined

# Task 7: Predictions

## 7.1: Plot Raw Predictions

Let's use the `compare_predictions` helper function to compare predictions from the model when it was untrained and when it was trained.

In [25]:
preds_on_trained = model.predict(x_test)
compare_predictions(preds_on_untrained, preds_on_trained, y_test)

NameError: name 'compare_predictions' is not defined

## 7.2: Plot Price Predictions

The plot for price predictions and raw predictions will look the same with just one difference: The x and y axis scale is changed.

In [ ]:
price_untrained = [convert_label_value(y) for y in preds_on_untrained]
price_trained = [convert_label_value(y) for y in preds_on_trained]
price_test = [convert_label_value(y) for y in y_test]

compare_predictions(price_untrained, price_trained, price_test)